In [ ]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 141kB/s 
     |████████████████████████████████| 14.5MB 321kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:


##Get data

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Just change the name of the searched file.

listed = drive.ListFile({'q': "title contains 'cropped_video06_images.zip'"}).GetList()
for cropped_file in listed:
  print('title {}, id {}'.format(cropped_file['title'], cropped_file['id']))

listed = drive.ListFile({'q': "title contains 'video06.zip'"}).GetList()
for full_file in listed:
  print('title {}, id {}'.format(full_file['title'], full_file['id']))

listed = drive.ListFile({'q': "title contains 'masks.zip'"}).GetList()
for mask_file in listed:
  print('title {}, id {}'.format(mask_file['title'], mask_file['id']))

title cropped_video06_images.zip, id 1nIiKNGaKKLR3X7oZpPuya_dG64-eofR1
title video06.zip, id 1lBvUJTAJPOLMrOFR4wkty4I91i9Un4Pk
title masks.zip, id 17p_dNMy4A_81udTTLuqbFg_XQ2lB4nS2


In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = cropped_file['id']

fname = os.path.join('/content', 'cropped_images.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)

file_id = full_file['id']

fname = os.path.join('/content', 'full_images.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)

file_id = mask_file['id']

fname = os.path.join('/content', 'masks.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)

downloading to /content/cropped_images.zip
downloading to /content/full_images.zip
downloading to /content/masks.zip


In [ ]:
!unzip cropped_images.zip
!unzip full_images.zip
!unzip masks.zip

Archive:  cropped_images.zip
   creating: cropped_video06_images/
  inflating: cropped_video06_images/anon012_26512.png  
  inflating: cropped_video06_images/anon012_26513.png  
  inflating: cropped_video06_images/anon012_26514.png  
  inflating: cropped_video06_images/anon012_26515.png  
  inflating: cropped_video06_images/anon012_26516.png  
  inflating: cropped_video06_images/anon012_26517.png  
  inflating: cropped_video06_images/anon012_26518.png  
  inflating: cropped_video06_images/anon012_26519.png  
  inflating: cropped_video06_images/anon012_26520.png  
  inflating: cropped_video06_images/anon012_26521.png  
  inflating: cropped_video06_images/anon012_26522.png  
  inflating: cropped_video06_images/anon012_26523.png  
  inflating: cropped_video06_images/anon012_26524.png  
  inflating: cropped_video06_images/anon012_26525.png  
  inflating: cropped_video06_images/anon012_26526.png  
  inflating: cropped_video06_images/anon012_26527.png  
  inflating: cropped_video06_images/an

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
cv2.ocl.setUseOpenCL(False)
import sys
from PIL import Image
from pathlib import Path
from skimage.metrics import structural_similarity as ssim
from os import listdir

Global Variables

In [ ]:
imgDirPath = '/content/cropped_video06_images'
boxPlotName = 'boxplot_video06_affine.png'
fullImgDirPath = '/content/video06'
mask_path = '/content/masks/anon012_mask.png'

padding_size = padding = 0
threshold = 7
showImages = False
transformation = "Affine"

In [ ]:
imgPaths =  [ imgDirPath + '/' + f  for f  in sorted(listdir(imgDirPath))]
fullImgPaths =  [ fullImgDirPath + '/' + f  for f  in sorted(listdir(fullImgDirPath))]

In [ ]:
img_1 = cv2.imread(fullImgPaths[0])
img_1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2RGB)

mask_im = Image.open(mask_path)
mask_im = mask_im.resize((img_1.shape[0], img_1.shape[1]), Image.ANTIALIAS)
mask_im = np.array(mask_im) * np.uint8(255)
mask_im = cv2.resize(mask_im, img_1.shape[1::-1])

In [ ]:
def inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages=True): 
  '''
    Inputs: srcImgPath, destImgPath 
    srcImgPath : path to the image been transformed, usually the new frame seen ->str
    destImgPath : path to the image that acts as destination(src image is trasformed to it), usually the old frame->str
    Output: [srcImg, destImg] -> list
    srcImg: 256x256 representation of srcImage ->ndarray
    destImg: 256x256 representation of destImage ->ndarray
  '''
  srcImg = cv2.imread(srcImgPath)
  srcImg = cv2.cvtColor(srcImg,cv2.COLOR_BGR2RGB)
  srcImg_gray = cv2.cvtColor(srcImg, cv2.COLOR_RGB2GRAY)

  destImg = cv2.imread(destImgPath)
  destImg = cv2.cvtColor(destImg,cv2.COLOR_BGR2RGB)
  destImg_gray = cv2.cvtColor(destImg, cv2.COLOR_RGB2GRAY)

  if showImages:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, constrained_layout=False, figsize=(10,4))
    ax1.imshow(destImg, cmap="gray")
    ax1.set_xlabel("dest image", fontsize=14)

    ax2.imshow(srcImg, cmap="gray")
    ax2.set_xlabel("Src image (Image to be transformed)", fontsize=14)
    
    plt.show()

  return [srcImg, destImg]

In [ ]:
def findTransformation(srcImg, destImg):
  '''
    ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
    ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
    threshold: ransac threshold. 
    Output:
    H - homography
    status - mask of ransac accepted or rejected
  '''

  mapper = cv2.reg_MapperGradAffine()
  mapperPyramid = cv2.reg_MapperPyramid(mapper)
  # mapperPyramid.numIterPerScale_ = 3
  # mapperPyramid.numLev_ = 3

  result_pointer = mapperPyramid.calculate(srcImg.astype(float), destImg.astype(float))
  result_array = cv2.reg.MapTypeCaster_toAffine(result_pointer)

  H = np.concatenate([result_array.getLinTr(), result_array.getShift()], axis = 1)
  status = False
  return [H, status]

In [ ]:
def visualizeStitch(srcImg, destImg, H, padding, showImages=True):
  '''
  srcImg: 256x256 representation of srcImage ->ndarray
  destImg: 256x256 representation of destImage ->ndarray
  H - homography
  '''
  ht, wd, cc= destImg.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  if transformation == "Homography":
    result = cv2.warpPerspective(srcImg, H, (ww, hh))
  elif transformation == "Affine":
    result = cv2.warpAffine(srcImg, H, (ww, hh))

  # result[yy:yy+ht, xx:xx+wd] = destImg

  if showImages:
    plt.figure(figsize=(10,4))
    plt.imshow(result)

    plt.show()

## Calculate all H values

In [ ]:
if transformation == "Homography":
  H_array = np.zeros((len(imgPaths)-1, 3,3 ))
elif transformation == "Affine":    
  H_array = np.zeros((len(imgPaths)-1, 2,3 ))
showImages = False

In [ ]:
for i in range(len(imgPaths) - 1):
  destImgPath = imgPaths[i] #the previous image
  srcImgPath =  imgPaths[i + 1];
  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages)
  H, status = findTransformation(srcImg, destImg)

  H_array[i] = H

In [ ]:
middle_num = len(fullImgPaths)//2
middle_num = middle_num - (middle_num%3)
middle_num

48

In [ ]:
!mkdir globalRegistrationTransparent

In [ ]:
def getHGlobal(H_array):
  H_global =  np.zeros((len(fullImgPaths), 3,3 ))

  for i in range(len(fullImgPaths)):
    # print(i)
    H_intermediate = np.eye(3)
    if i < middle_num:
      for j in range(i, middle_num):
         H_intermediate = np.matmul( H_intermediate,
                                    np.vstack([H_array[j], [0,0,1]]) ) 
      H_intermediate = np.linalg.inv(H_intermediate)
    elif i > middle_num:
      for j in range(middle_num, i):
         H_intermediate = np.matmul( H_intermediate,
                                    np.vstack([H_array[j], [0,0,1]]) )

    #note that if i == middle_num, we use unity array
    H_global[i] = H_intermediate

  return H_global

In [ ]:
padding_size = padding = 2500

In [ ]:
H_global = getHGlobal(H_array)

In [ ]:
H_global[: , 0:2, 2] = H_global[: , 0:2, 2] + [padding_size]

In [ ]:
def getTransparentImg(src, imageName):
  tmp = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
  _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
  r, g, b = cv2.split(src)
  rgba = [b,g,r, alpha]
  dst = cv2.merge(rgba,4)
  save_name = "/content/globalRegistrationTransparent/" + imageName
  cv2.imwrite(save_name, dst)

In [ ]:
def globalImageRegistration(srcImg, destImg, index, imageName, padding=padding_size):
  ht, wd, cc= destImg.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  srcImg = cv2.bitwise_and(srcImg, srcImg, mask=mask_im) 

  result = cv2.warpPerspective(srcImg, H_global[index], (ww, hh))

  return result

In [ ]:
for i in range(len(fullImgPaths)):
  destImgPath = fullImgPaths[middle_num] #the first image
  srcImgPath =  fullImgPaths[i];
  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, False)

  imageName = Path(fullImgPaths[i]).name
  src = globalImageRegistration(srcImg, destImg, i, imageName)
  
  getTransparentImg(src, imageName)

In [ ]:
!zip -r /content/globalRegistration.zip /content/globalRegistrationTransparent

  adding: content/globalRegistrationTransparent/ (stored 0%)
  adding: content/globalRegistrationTransparent/anon012_26566.png (deflated 49%)
  adding: content/globalRegistrationTransparent/anon012_26550.png (deflated 37%)
  adding: content/globalRegistrationTransparent/anon012_26515.png (deflated 24%)
  adding: content/globalRegistrationTransparent/anon012_26575.png (deflated 53%)
  adding: content/globalRegistrationTransparent/anon012_26578.png (deflated 54%)
  adding: content/globalRegistrationTransparent/anon012_26597.png (deflated 50%)
  adding: content/globalRegistrationTransparent/anon012_26576.png (deflated 51%)
  adding: content/globalRegistrationTransparent/anon012_26537.png (deflated 28%)
  adding: content/globalRegistrationTransparent/anon012_26529.png (deflated 26%)
  adding: content/globalRegistrationTransparent/anon012_26601.png (deflated 51%)
  adding: content/globalRegistrationTransparent/anon012_26604.png (deflated 54%)
  adding: content/globalRegistrationTransparent/

In [ ]:
import shutil
shutil.rmtree('/content/globalRegistrationTransparent')